In [52]:
import ee
import pandas as pd
import geopandas as gpd
import math
from datetime import datetime
from datetime import timezone

ee.Initialize()

def download_met(lon, lat, time):
    """
    Retrieves  data from the ERA5-Land hourly model using Google Earth Engine https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY

    The standard units are converted to the units required by the GLM model.

    The output csv file is ready to be used for running the GLM model,    


    Parameters
    ----------
    geofile (str) -- path to the geojson file containing the name and coordinates of the site. 
      
    start (str) -- starting date of the data request in the form YYYY-MM-DD
    
    end (str) -- ending date areaof the data request in the form YYYY-MM-DD

    Returns
    -------
    Nothing

    The output csv file is saved in the current directory.


    """

    geo = create_geo(lon, lat)
    SatImage = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select(
        [
            "temperature_2m",
            "dewpoint_temperature_2m",
            "u_component_of_wind_10m",
            "v_component_of_wind_10m",
            "total_precipitation_hourly",
            "leaf_area_index_high_vegetation",
            "soil_temperature_level_1",
        ]
    )
    ogtime = time
    start = time + "T12:00:00Z"
    end = time + "T13:00:00Z"
    start = datetime.strptime(start, '%Y-%m-%dT%H:%M:%S%z')
    end = datetime.strptime(end, '%Y-%m-%dT%H:%M:%S%z')
    start = start.strftime('%Y-%m-%dT%H:%M:%S%z')
    end = end.strftime('%Y-%m-%dT%H:%M:%S%z')
    SatImageTime = SatImage.filterDate(start, end)
    info = SatImageTime.filterBounds(geo).getRegion(geo, 500).getInfo()
    info_dict = [dict(zip(info[0], values)) for values in info[1:]]
    df = pd.DataFrame(info_dict)        
    AirTemp = df["temperature_2m"].apply(converto_c)[0]
    SoilTemp = df["soil_temperature_level_1"].apply(converto_c)[0]
    LAI = df["leaf_area_index_high_vegetation"][0]
    Rain = df["total_precipitation_hourly"][0]
    dewpoint_temperature_2m = df["dewpoint_temperature_2m"].apply(converto_c)[0]
    RelHum = df.apply(
        lambda x: calculate_rh(x["temperature_2m"], x["dewpoint_temperature_2m"]),
        axis=1,
    )[0]
    WindSpeed = df.apply(
        lambda x: calculate_windspeed(
            x["u_component_of_wind_10m"], x["v_component_of_wind_10m"]
        ),
        axis=1,
    )[0]
    row = {'X':lon, 'Y':lat, 'FireDiscoveryDateTime':ogtime, "AirTemp":AirTemp, "dewpoint_temperature_2m":dewpoint_temperature_2m, "RelHum":RelHum, "WindSpeed":WindSpeed, "Rain":Rain, "SoilTemp":SoilTemp, "LAI":LAI}
    return row


def create_geo(lon, lat):
    geo = ee.Geometry.Point(lon, lat)
    return geo


def unixto_utc(timestamp):
    timestamp = str(timestamp)
    timestamp = timestamp[:-3]
    timestamp = int(timestamp)
    dt_object = datetime.fromtimestamp(timestamp, tz=timezone.utc)
    return dt_object.strftime("%Y-%m-%d %H:%M:%S")



def converto_c(temp=0):
    if temp == None:
        print(temp, "check")
        temp = 999
    return temp - 273.15


def calculate_rh(temp=0, dewpoint=0):
    """
    https://en.wikipedia.org/wiki/Dew_point?oldformat=true
    https://earthscience.stackexchange.com/questions/16570/how-to-calculate-relative-humidity-from-temperature-dew-point-and-pressure
    """
    if temp == None:
        print(temp, "check")
        temp = 999
    if dewpoint == None:
        print(dewpoint, "check")
        dewpoint = 999
    rh = 100 - (5 * (temp - dewpoint))
    return rh


def calculate_windspeed(u_comp=0, v_comp=0):
    if u_comp == None:
        print(u_comp, "check")
        u_comp = 999
    if v_comp == None:
        print(v_comp, "check")
        v_comp = 999
    ws = math.sqrt((u_comp * u_comp) + (v_comp * v_comp))
    return ws

In [53]:
a = download_met(lat=37.067843, lon=-119.492714, time="2022-06-01")

In [54]:
a

{'X': -119.492714,
 'Y': 37.067843,
 'FireDiscoveryDateTime': '2022-06-01',
 'AirTemp': 10.798181152343773,
 'dewpoint_temperature_2m': 3.5837188720703352,
 'RelHum': 63.92768859863281,
 'WindSpeed': 2.142577214032191,
 'Rain': 0,
 'SoilTemp': 12.115151977539085,
 'LAI': 2.1092529296875}

In [55]:
ca = pd.read_csv('ca-fires-all.csv')

In [56]:
ca['FireDiscoveryDateTime'] = pd.to_datetime(ca['FireDiscoveryDateTime'])
ca['FireDiscoveryDateTime'] = ca['FireDiscoveryDateTime'].dt.strftime('%Y-%m-%d')

In [57]:
ca = ca[(ca["FireDiscoveryDateTime"] >"2016-01-01")]
ca = ca[(ca["FireDiscoveryDateTime"] <"2022-06-01")]

In [58]:
ca

,X,Y,FireDiscoveryDateTime,InitialLatitude,InitialLongitude
4737,-117.841012,34.173305,2016-01-03,NaN,NaN
4738,-118.455012,34.663905,2016-01-04,NaN,NaN
4739,-116.430012,32.844506,2016-01-06,32.844501,-116.430000
4740,-116.642109,32.779905,2016-01-06,32.779900,-116.642097
4741,-123.508017,41.356607,2016-01-07,41.308601,-123.542801
...,...,...,...,...,...
50244,-122.268819,38.371644,2022-05-31,38.371639,-122.268806
50245,-116.514612,33.980869,2022-05-31,33.980864,-116.514600
50246,-122.280116,41.230649,2022-05-31,41.230644,-122.280102
50247,-120.483419,37.832427,2022-05-31,37.832422,-120.483406


In [59]:
ca = ca.sample(1000)

In [60]:
ca

,X,Y,FireDiscoveryDateTime,InitialLatitude,InitialLongitude
44010,-117.148517,33.502941,2021-10-25,33.502936,-117.148505
13784,-120.499113,39.618525,2018-08-07,39.565740,-120.632800
27424,-120.726713,41.783735,2020-07-24,41.790160,-120.729200
49420,-120.828646,37.313022,2022-05-18,37.313017,-120.828633
46558,-122.310314,41.576222,2022-02-23,41.576217,-122.310300
...,...,...,...,...,...
47866,-118.066992,34.006195,2022-04-08,34.006190,-118.066980
44974,-121.133787,36.200235,2021-12-04,36.200230,-121.133774
35666,-120.636113,39.149195,2021-05-18,39.145660,-120.640000
12708,-120.558902,35.082227,2018-06-30,NaN,NaN


In [61]:
new_ca = pd.DataFrame(columns=["X", "Y", "FireDiscoveryDateTime",
       'AirTemp', 'dewpoint_temperature_2m', 'RelHum', 'WindSpeed',
       'Rain', 'SoilTemp', 'LAI'
])

In [62]:
%%time
for index, row in ca.iterrows():
    dat = download_met(lat=row['Y'], lon=row['X'], time=row['FireDiscoveryDateTime'])
    new_ca = new_ca.append(dat, ignore_index=True)

None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
None check
CPU times: user 39.1 s, sys: 243 ms, total: 39.3 s
Wall time: 12min 57s


In [63]:
new_ca.to_csv('fire_met.csv')

In [64]:
test = pd.read_csv('fire_met.csv')

In [65]:
test

,Unnamed: 0,X,Y,FireDiscoveryDateTime,AirTemp,dewpoint_temperature_2m,RelHum,WindSpeed,Rain,SoilTemp,LAI
0,0,-117.148517,33.502941,2021-10-25,11.625513,11.388345,98.814163,1.371513,8.000965e-06,12.534158,4.441162
1,1,-120.499113,39.618525,2018-08-07,14.219232,2.894510,43.376389,1.201769,0.000000e+00,13.674631,2.954102
2,2,-120.726713,41.783735,2020-07-24,13.663507,9.753458,80.449753,1.885694,0.000000e+00,16.071771,2.158691
3,3,-120.828646,37.313022,2022-05-18,15.576730,6.861505,56.423874,1.977814,0.000000e+00,18.434167,1.927734
4,4,-122.310314,41.576222,2022-02-23,-7.894141,-15.418051,62.380447,1.520608,0.000000e+00,-0.932135,4.300049
...,...,...,...,...,...,...,...,...,...,...,...
995,995,-118.066992,34.006195,2022-04-08,14.627863,-0.415839,24.781494,2.876088,0.000000e+00,14.263940,4.822998
996,996,-121.133787,36.200235,2021-12-04,8.651178,5.419534,83.841782,1.121236,0.000000e+00,7.676614,2.106445
997,997,-120.636113,39.149195,2021-05-18,7.055551,5.099435,90.219421,1.388955,0.000000e+00,8.941400,2.411621
998,998,-120.558902,35.082227,2018-06-30,13.775995,12.453638,93.388214,2.793564,-7.450581e-09,15.169870,2.676636
